In [1]:
import psycopg2

In [2]:
#Função para criar tabela

def create_database():
    #Conexão com o database padrão do PostgreSQL
    conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=Gemeos$17")
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    #Criando um database com formato UTF8
    cur.execute("DROP DATABASE movies")
    cur.execute("CREATE DATABASE movies")

    #Encerrando a sessão atual
    conn.close()

    #Conectando ao novo database
    conn = psycopg2.connect("host=localhost dbname=movies user=postgres password=Gemeos$17")
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    return cur, conn

In [2]:
"""conn = psycopg2.connect("host=localhost dbname=movies user=postgres password=Gemeos$17")
conn.set_session(autocommit=True)
cur = conn.cursor()"""

In [24]:
"""def drop_tables(cur, conn):
    for query in drop_tables_queries:
        cur.execute(query)
        conn.commit()"""

In [25]:
"""def create_tables(cur, conn):
    for query in create_tables_queries:
        cur.execute(query)
        conn.commit()"""

In [41]:
def read_csv(arquivo):

    with open(arquivo, 'r') as arquivo_csv:
        # Lendo todas as linhas do arquivo
        linhas = arquivo_csv.readlines()

    # Criando um dicionário vazio para armazenar os dados
    lista = []

    # Obtendo os cabeçalhos (a primeira linha do arquivo)
    cabecalhos = linhas[0].strip().split(',')

    # Iterando sobre as linhas do arquivo, excluindo o cabeçalho
    for linha in linhas[1:]:
        # Dividindo a linha em campos usando a vírgula como separador
        campos = linha.strip().split(',')
        
        # Criando um dicionário para armazenar os valores desta linha
        valores = {}
        for i, cabecalho in enumerate(cabecalhos):
            
            valores[cabecalho] = campos[i] if campos[i] else None
                
        # Adicionando ao dicionário principal
        lista.append(valores)

    # Exibindo o dicionário resultante
    return lista

In [42]:
moviesdb = read_csv("movies.csv")
genome_scoresdb = read_csv("genome-scores.csv")
linksdb = read_csv("links.csv")

In [57]:
conn.close()

In [58]:
cur, conn = create_database()

In [59]:
movies_table_create = ("""CREATE TABLE IF NOT EXISTS movies(
movieId INT PRIMARY KEY,
title VARCHAR,
genres VARCHAR
)""")
cur.execute(movies_table_create)
conn.commit()

genome_scores_table_create = ("""CREATE TABLE IF NOT EXISTS genome_scores(
movieId INT,
tagId INT,
relavance NUMERIC
)""")
cur.execute(genome_scores_table_create)
conn.commit()

links_table_create = ("""CREATE TABLE IF NOT EXISTS links(
movieId INT,
imdbId INT NULL,
tmdbId INT NULL
)""")
cur.execute(links_table_create)
conn.commit()

In [60]:
movies_table_insert = ("""INSERT INTO movies(
movieId,
title,
genres)
VALUES (%s, %s, %s)
""")

genome_scores_table_insert = ("""INSERT INTO genome_scores(
movieId,
tagId,
relavance)
VALUES (%s, %s, %s)
""")

links_table_insert = ("""INSERT INTO links(
movieId,
imdbId,
tmdbId)
VALUES (%s, %s, %s)
""")

In [49]:
for item in moviesdb:
    cur.execute(movies_table_insert, list(item.values()))

In [48]:
for item in linksdb:
    cur.execute(links_table_insert, list(item.values()))

In [68]:
#Esse arquivo tem 15.5 milhões de linhas, vou deixar do mesmo tamanho dos outroas
valores_lista = []
for item in genome_scoresdb[:60000]:
    valores_lista.append(tuple(item.values()))

In [69]:
cur.executemany(genome_scores_table_insert, valores_lista)